# Calculating travel time to ports based on select road improvements in Tunisia

Unit of analysis is ADM2 (geoboundaries). Need to summarize the following:
- Monthly Nighttime Lights mean and sum
- TT to ports pre and post implementation of TREATED roads
- Distance to nearest treated road, distance to each labelled treated road
- Percentage Urban

In [1]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd
import skimage.graph as graph
import numpy as np

from shapely.geometry import Point

sys.path.insert(0, r"C:\WBG\Work\Code\GOSTrocks\src")
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.osmMisc as osmMisc
import GOSTrocks.dataMisc as dMisc
import GOSTrocks.ntlMisc as ntlMisc
from GOSTrocks.misc import tPrint

sys.path.append(r"C:\WBG\Work\Code\GOSTnetsraster\src")
import GOSTnetsraster.market_access as ma
import GOSTnetsraster.conversion_tables as speed_tables

%load_ext autoreload
%autoreload 2

In [2]:
# Input parameters
m_crs = 32632 # Need to project data to a metres-based projection

# Define input data
base_folder = "C:/WBG/Work/TUN_Impact/"
landcover_file = os.path.join(base_folder, "DATA", 'ESA_Globcover.tif')
# These are the digitized road segements that have been improved
treated_segments_file = os.path.join(base_folder, "DATA", "DIME_Roads", 'treated_roads.shp')
control_segments_file = os.path.join(base_folder, "DATA", "DIME_Roads", 'control_roads.shp')
road_segments_file = os.path.join(base_folder, "DATA", "impacted_osm_roads.gpkg")
osm_roads_file = os.path.join(base_folder, "DATA", "OSM", "gis_osm_roads_free_1.shp")
# WorldPop 2020 constrained, projected to m_crs
pop_file = os.path.join(base_folder, "DATA", "tun_ppp_2020_constrained_proj.tif")
# https://datacatalog.worldbank.org/int/search/dataset/0038118/Global---International-Ports
port_file = os.path.join(base_folder, "DATA", "TUN_ports.gpkg")
# administrative bounadaries are used to summarize population
tun_adm2 = dMisc.get_geoboundaries("TUN", 'ADM2')
tun_adm1 = dMisc.get_geoboundaries("TUN", 'ADM1')

# Define output files
friction_folder = os.path.join(base_folder, "DATA", "FRICTION")
results_folder = os.path.join(base_folder, "RESULTS")
for cFolder in [friction_folder, results_folder]:
    if not os.path.exists(cFolder):
        os.makedirs(cFolder)    
pre_friction_file = os.path.join(friction_folder, 'FRICTION_pre_intervention.tif')
post_friction_file = os.path.join(friction_folder, 'FRICTION_post_intervention.tif')
# This extracts the existing global friction file, used only for comparison
global_friction_file = os.path.join(friction_folder, "2020_motorized_friction.geotiff")
if not os.path.exists(global_friction_file):
    gl_fr = rasterio.open(r"J:\Data\GLOBAL\INFRA\FRICTION_2020\2020_motorized_friction_surface.geotiff")
    local_fr = rMisc.clipRaster(gl_fr, tun_adm2, global_friction_file)
    
# Read in data
dests = gpd.read_file(port_file).to_crs(m_crs)
if not os.path.exists(landcover_file):
    global_landcover = r"R:\GLOBAL\LCVR\Globcover\2015\ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
    in_lc = rasterio.open(global_landcover)
    temp_landcover_file = landcover_file.replace(".tif", "_temp.tif")
    local_lc = rMisc.clipRaster(in_lc, tun_adm2, temp_landcover_file)
    temp_lc = rasterio.open(temp_landcover_file)
    proj_res = rMisc.project_raster(temp_lc, m_crs)
    with rasterio.open(landcover_file, 'w', **proj_res[1]) as outR:
        outR.write(proj_res[0])

in_lc = rasterio.open(landcover_file)
in_pop = rasterio.open(pop_file)
if in_pop.crs != in_lc.crs:
    proj_res = rMisc.standardizeInputRasters(in_pop, in_lc, pop_file.replace(".tif", "_proj.tif"))

In [ ]:
# Downlaod worldcover data
tiles_geojson = r"C:\WBG\Work\data\LCVR\esa_worldcover_grid.geojson"
in_tiles = gpd.read_file(tiles_geojson)
sel_tiles = in_tiles.loc[in_tiles.intersects(tun_adm2.unary_union)]

tile_path = "s3://esa-worldcover/v200/2021/map/ESA_WorldCover_10m_2021_v200_{tile}_Map.tif"
out_folder = os.path.join(base_folder, "DATA", "WorldCover")
for idx, row in sel_tiles.iterrows():
    cur_tile_path = tile_path.format(tile=row['ll_tile'])
    cur_out = os.path.join(out_folder, f"WorldCover_{row['ll_tile']}.tif")
    if not os.path.exists(cur_out):
        command = f"aws s3 --no-sign-request --no-verify-ssl cp {cur_tile_path} {cur_out}"
        print(command)


In [ ]:
# Download and process industrial zones
import urllib.request, json 
with urllib.request.urlopen("https://afi.e-industrie.gov.tn/apps-lots.php") as url:
    industry_zones = json.load(url)
zones_df = pd.DataFrame(industry_zones)

zones_geoms = [Point(x) for x in zip(zones_df['ZONE_LONG'], zones_df['ZONE_LAT'])]
zones_df = gpd.GeoDataFrame(zones_df, geometry=zones_geoms, crs=4326)
zones_df = zones_df.to_crs(m_crs)
zones_df.to_file(os.path.join(base_folder, "DATA", "MAPPING", "industrial_zones.gpkg"), driver='GPKG')

In [ ]:
# Process roads to create pre and post friction surfaces
sel_roads = gpd.read_file(road_segments_file)
sel_roads = sel_roads.to_crs(m_crs)
sel_roads['speed'] = 10
all_roads = gpd.read_file(osm_roads_file)
all_roads = all_roads.to_crs(m_crs)
all_roads['speed'] = all_roads['fclass'].map(speed_tables.osm_speed_dict)
all_roads['speed'] = all_roads['speed'].fillna(10.0)
wb_roads_ids = sel_roads.loc[~sel_roads['osm_id'].isna(),'osm_id']
new_roads = sel_roads.loc[sel_roads['osm_id'].isna(),]

lc_speed_table = speed_tables.esaacci_landcover

In [ ]:
# Generate pre-intervention friction surface
if not os.path.exists(pre_friction_file):
    pre_roads = all_roads.copy()
    pre_roads.loc[pre_roads['osm_id'].isin(wb_roads_ids), 'speed'] = 10.0

    pre_friction = ma.generate_roads_lc_friction(in_lc, pre_roads, lc_travel_table=lc_speed_table, 
                             out_file=pre_friction_file, resolution=in_lc.res[0])

pre_friction = rasterio.open(pre_friction_file)

In [ ]:
# Generate post-intervention friction surface
if not os.path.exists(post_friction_file):
    post_roads = all_roads.copy()
    post_roads.loc[post_roads['osm_id'].isin(wb_roads_ids), 'speed'] = 40.0

    post_friction = ma.generate_roads_lc_friction(in_lc, post_roads, lc_travel_table=lc_speed_table, 
                              out_file=post_friction_file, resolution=in_lc.res[0])
    
post_friction = rasterio.open(post_friction_file)

# Calculate travel time

In [ ]:
# Calculate pre-intervention, population-weighted travel time
frictionD = pre_friction.read()[0,:,:]
frictionD = frictionD * pre_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
pre_tt_ports = ma.summarize_travel_time_populations(in_pop, pre_friction, dests, mcp, tun_adm2)
pd.DataFrame(pre_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM2_tt_ports.csv"))

pre_zones_ports = ma.summarize_travel_time_populations(in_pop, pre_friction, zones_df, mcp, tun_adm2)
pd.DataFrame(pre_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM2_tt_zones.csv"))

In [ ]:
# Calculate pre-intervention, population-weighted travel time
frictionD = pre_friction.read()[0,:,:]
frictionD = frictionD * pre_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
pre_tt_ports = ma.summarize_travel_time_populations(in_pop, pre_friction, dests, mcp, tun_adm1)
pd.DataFrame(pre_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM1_tt_ports.csv"))

pre_zones_ports = ma.summarize_travel_time_populations(in_pop, pre_friction, zones_df, mcp, tun_adm1)
pd.DataFrame(pre_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM1_tt_zones.csv"))

In [ ]:
# Calculate pre-intervention, population-weighted travel time
frictionD = post_friction.read()[0,:,:]
frictionD = frictionD * post_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
post_tt_ports = ma.summarize_travel_time_populations(in_pop, post_friction, dests, mcp, tun_adm2)
pd.DataFrame(post_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM2_tt_ports.csv"))

post_zones_ports = ma.summarize_travel_time_populations(in_pop, post_friction, zones_df, mcp, tun_adm2)
pd.DataFrame(post_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM2_tt_zones.csv"))

In [ ]:
# Calculate pre-intervention, population-weighted travel time
frictionD = post_friction.read()[0,:,:]
frictionD = frictionD * post_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
post_tt_ports = ma.summarize_travel_time_populations(in_pop, post_friction, dests, mcp, tun_adm1)
pd.DataFrame(post_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM1_tt_ports.csv"))

post_zones_ports = ma.summarize_travel_time_populations(in_pop, post_friction, zones_df, mcp, tun_adm1)
pd.DataFrame(post_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM1_tt_zones.csv"))

# Zonal stats on nighttimelights

In [3]:
ntl_files = ntlMisc.aws_search_ntl()

c:\wbg\Anaconda3\envs\gn\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'globalnightlight.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
# Clip out NTL raster files for pre and post
pre_file = [x for x in ntl_files if "201501" in x][0]
post_file = [x for x in ntl_files if "202401" in x][0]

ntl_out_folder = os.path.join(base_folder, "DATA", "NTL_Rasters")
with rasterio.Env(GDAL_HTTP_UNSAFESSL = 'YES') as env:
    pre_res = rMisc.clipRaster(rasterio.open(pre_file), tun_adm1, os.path.join(ntl_out_folder, "VIIRS_201501.tif"))
    post_res = rMisc.clipRaster(rasterio.open(post_file), tun_adm1, os.path.join(ntl_out_folder, "VIIRS_202401.tif"))

In [7]:
ntl_diff = post_res[0] - pre_res[0]
with rasterio.open(os.path.join(ntl_out_folder, "VIIRS_201501_202401.tif"), 'w', **pre_res[1]) as outR:
    outR.write(ntl_diff)

# Measure distance to treated roads

In [ ]:
tPrint("Start")
for road_file, out_file, col_lbl in [
            [treated_segments_file, "distance_to_treated_roads.csv", "RTE_NOM"],
            [control_segments_file, "distance_to_control_roads.csv", "RTE_NOM"],
            [road_segments_file, "distance_to_WB_digitized_roads.csv", "road_group"],
                    ]:
    roads = gpd.read_file(road_file)
    roads = roads.to_crs(m_crs)
    tun_adm2 = tun_adm2.to_crs(m_crs)
    for lbl, df in roads.groupby(col_lbl):
        tun_adm2[f'dist_{lbl}'] = tun_adm2.apply(lambda x: x["geometry"].distance(df.union_all()), axis=1)
    tun_adm2[f'dist_road'] = tun_adm2.apply(lambda x: x["geometry"].distance(roads.union_all()), axis=1)
    pd.DataFrame(tun_adm2.drop(['geometry'], axis=1)).to_csv(os.path.join(base_folder, "RESULTS", out_file))
    tPrint(out_file)



In [ ]:
gpd.read_file(control_segments_file)['RTE_NOM'].unique()

# PRepare mapping data

In [ ]:
results_csv = os.path.join(base_folder, "DATA", "MAPPING", "mapping_res.csv")
in_res = pd.read_csv(results_csv)

In [ ]:
pd.merge(tun_adm1, in_res, left_on="shapeName", right_on='adm1', how='outer').to_file( 
    os.path.join(base_folder, "DATA", "MAPPING", "adm1_res.gpkg"), driver="GPKG")

In [ ]:
pd.merge(tun_adm1, in_res, left_on="shapeName", right_on='adm1', how='outer').sort_values('time_hub')